In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
from sklearn import preprocessing
import os
import seaborn as sns
from skimage.feature import hog

- Trajectory:    2x[trajectory length] (default 30 dimension)
- HOG:           8x[spatial cells]x[spatial cells]x[temporal cells] (default 96 dimension)
- HOF:           9x[spatial cells]x[spatial cells]x[temporal cells] (default 108 dimension)
- MBHx:          8x[spatial cells]x[spatial cells]x[temporal cells] (default 96 dimension)
- MBHy:          8x[spatial cells]x[spatial cells]x[temporal cells] (default 96 dimension)

In [2]:
trajectory_range = 40
hog_range = trajectory_range+96
hof_range = hog_range+108
mbhx_range = hof_range+96
mbhy_range = mbhx_range+96

### Separating files for training and testing

In [3]:
feature_files_path="./features/"
files=[os.path.join(feature_files_path,file) for file in os.listdir(feature_files_path)]

In [4]:
training_files = ["-nm-01-","-nm-02-","-nm-03-","-nm-04-"]
training_set_files = [file for file in files if any(training_file in file for training_file in training_files)]

In [5]:
test_1_files = ["-nm-05-","-nm-06-"]
test_set_1_files = [file for file in files if any(training_file in file for training_file in test_1_files)]

In [6]:
test_2_files = ["-bg-01-","-bg-02-"]
test_set_2_files = [file for file in files if any(training_file in file for training_file in test_2_files)]

In [7]:
test_3_files = ["-cl-01-","-cl-02-"]
test_set_3_files = [file for file in files if any(training_file in file for training_file in test_3_files)]

## Codebook for training data

In [8]:
#hog = []
#mbhx = []
#mbhy = []
#for file in training_set_files:
    
    #features = pd.read_csv(file,'\t',header=None)
    #features.drop(features.columns[-1],axis=1,inplace=True)
    
    #hog.append(features.iloc[:,trajectory_range:hog_range])
    #mbhx.append(features.iloc[:,hof_range:mbhx_range])
    #mbhy.append(features.iloc[:,mbhx_range:mbhy_range])

In [9]:
#print(len(hog))
#print(len(mbhx))
#print(len(mbhy))

496
496
496


In [10]:
#hog_df = pd.DataFrame()
#mbhx_df = pd.DataFrame()
#mbhy_df = pd.DataFrame()

#for i in range(len(hog)):    
#    hog_df = hog_df.append(hog[i])
    
#for i in range(len(mbhx)):    
#    mbhx_df = mbhx_df.append(mbhx[i])
    
#for i in range(len(mbhy)):    
#    mbhy_df = mbhy_df.append(mbhy[i])

In [15]:
#print(hog_df.shape)
#print(mbhx_df.shape)
#print(mbhy_df.shape)

(2969163, 96)
(2969163, 96)
(2969163, 96)


In [16]:
#hog_df.to_csv('hog_features_training.csv')
#mbhx_df.to_csv('mbhx_features_training.csv')
#mbhy_df.to_csv('mbhy_features_training.csv')

## HOG Codebook

In [8]:
hog = pd.read_csv('hog_features_training.csv')
hog = hog.drop(hog.columns[0],axis=1)

In [10]:
mbhx = pd.read_csv('mbhx_features_training.csv')
mbhy = pd.read_csv('mbhy_features_training.csv')

,Unnamed: 0,340,341,342,343,344,345,346,347,348,...,426,427,428,429,430,431,432,433,434,435
0,0,0.206970,0.230470,0.178903,0.208676,0.225700,0.243112,0.220176,0.237327,0.158412,...,0.126315,0.168695,0.129891,0.192684,0.219250,0.183096,0.175372,0.236093,0.218617,0.194773
1,1,0.241674,0.255948,0.244432,0.261008,0.210541,0.274907,0.266184,0.259806,0.157818,...,0.120017,0.184843,0.248799,0.186563,0.162074,0.208596,0.206460,0.206046,0.198800,0.262517
2,2,0.089144,0.125173,0.080656,0.079282,0.102164,0.140414,0.094191,0.073781,0.176921,...,0.101027,0.124621,0.297401,0.267900,0.214632,0.187455,0.178844,0.232761,0.249239,0.336545
3,3,0.130290,0.149805,0.094081,0.118641,0.156127,0.162456,0.115063,0.112889,0.152060,...,0.145189,0.177442,0.232718,0.223823,0.243018,0.230713,0.158559,0.216203,0.237785,0.282377
4,4,0.057187,0.110860,0.150124,0.094627,0.062576,0.126829,0.153000,0.111466,0.264008,...,0.150986,0.153212,0.238432,0.268040,0.208385,0.174218,0.201569,0.226218,0.263132,0.285877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969158,5785,0.124658,0.119276,0.178761,0.267656,0.222673,0.200233,0.138065,0.173611,0.108961,...,0.130887,0.117670,0.076563,0.099385,0.085017,0.089424,0.083933,0.083831,0.101702,0.103382
2969159,5786,0.207487,0.246775,0.243528,0.148294,0.084284,0.084687,0.102175,0.183292,0.246654,...,0.100687,0.100814,0.097433,0.103770,0.107351,0.113586,0.088066,0.071316,0.100535,0.125664
2969160,5787,0.148017,0.186114,0.226385,0.271672,0.240654,0.258676,0.265062,0.253761,0.114557,...,0.114003,0.123371,0.095034,0.117017,0.112128,0.112683,0.094179,0.103577,0.115726,0.121262
2969161,5788,0.215675,0.244842,0.252596,0.193926,0.203575,0.148289,0.105967,0.159822,0.200358,...,0.091240,0.095123,0.068924,0.080188,0.084495,0.073272,0.070149,0.088850,0.104165,0.090657


In [12]:
#
mbhx = mbhx.drop(mbhx.columns[0],axis=1)
mbhy = mbhy.drop(mbhy.columns[0],axis=1)

MemoryError: Unable to allocate 2.12 GiB for an array with shape (96, 2969163) and data type float64